In [12]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Define customer and module characteristics
customers = [
    {"id": 1, "modules": [1, 2, 3]},
    {"id": 2, "modules": [1, 4, 5]},
    {"id": 3, "modules": [2, 3, 4]},
    {"id": 4, "modules": [6, 6]}  # Two requests for module 6
]

module_servers = {
    1: 2,
    2: 2,
    3: 2,
    4: 1,
    5: 1,
    6: 1
}

# Initialize simulation variables
current_time = datetime.now()
output_data = []

class Server:
    def __init__(self):
        self.busy_until = current_time

servers = {
    1: Server(),
    2: Server(),
    3: Server(),
    4: Server(),
    5: Server(),
    6: Server()
}

# Define service time distributions (mean and std deviation)
module_service_distributions = {
    1: {"mean": 3, "std_dev": 1},
    2: {"mean": 3, "std_dev": 1},
    3: {"mean": 3, "std_dev": 1},
    4: {"mean": 4, "std_dev": 2},
    5: {"mean": 4, "std_dev": 2},
    6: {"mean": 10, "std_dev": 3}
}

# Simulate the queue and service
for customer in customers:
    for module in customer["modules"]:
        while servers[module].busy_until > current_time:
            current_time += timedelta(seconds=1)

        start_time = current_time

        # Generate random service time based on the normal distribution
        service_time = max(np.random.normal(module_service_distributions[module]["mean"],
                                            module_service_distributions[module]["std_dev"]), 0.5)

        end_time = current_time + timedelta(minutes=service_time)

        output_data.append({
            "Customer ID": customer["id"],
            "Module": module,
            "Start Time": start_time,
            "End Time": end_time,
            "Server": f"Server {module_servers[module]}"
        })

        servers[module].busy_until = end_time
        current_time = end_time

# Create DataFrame and Excel output
df = pd.DataFrame(output_data)
file_path = "queue_output.xlsx"
df.to_excel(file_path, index=False)
print(f"Output saved to {file_path}")

Output saved to queue_output.xlsx


In [13]:
# Incluye las solicitudes que se atendieron

import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Define customer and module characteristics
customers = [
    {"id": 1, "modules": [1, 2, 3]},
    {"id": 2, "modules": [1, 4, 5]},
    {"id": 3, "modules": [2, 3, 4]},
    {"id": 4, "modules": [6, 6]}  # Two requests for module 6
]

module_servers = {
    1: 2,
    2: 2,
    3: 2,
    4: 1,
    5: 1,
    6: 1
}

# Initialize simulation variables
current_time = datetime.now()
output_data = []

class Server:
    def __init__(self):
        self.busy_until = current_time
        self.current_request = None

servers = {
    1: Server(),
    2: Server(),
    3: Server(),
    4: Server(),
    5: Server(),
    6: Server()
}

# Define service time distributions (mean and std deviation)
module_service_distributions = {
    1: {"mean": 3, "std_dev": 1},
    2: {"mean": 3, "std_dev": 1},
    3: {"mean": 3, "std_dev": 1},
    4: {"mean": 4, "std_dev": 2},
    5: {"mean": 4, "std_dev": 2},
    6: {"mean": 10, "std_dev": 3}
}

# Simulate the queue and service
for customer in customers:
    for module in customer["modules"]:
        while servers[module].busy_until > current_time:
            current_time += timedelta(seconds=1)

        start_time = current_time

        # Generate random service time based on the normal distribution
        service_time = max(np.random.normal(module_service_distributions[module]["mean"],
                                            module_service_distributions[module]["std_dev"]), 0.5)

        end_time = current_time + timedelta(minutes=service_time)

        if customer["id"] == 4:
            if servers[module].current_request is None:
                servers[module].current_request = 1
            else:
                servers[module].current_request = 2

        output_data.append({
            "Customer ID": customer["id"],
            "Module": module,
            "Start Time": start_time,
            "End Time": end_time,
            "Server": f"Server {module_servers[module]}",
            "Request": servers[module].current_request if customer["id"] == 4 else None
        })

        servers[module].busy_until = end_time
        current_time = end_time

# Create DataFrame and Excel output
df = pd.DataFrame(output_data)
file_path = "queue_output.xlsx"
df.to_excel(file_path, index=False)
print(f"Output saved to {file_path}")

Output saved to queue_output.xlsx


In [15]:
# Codigo con restriccion en los servidores: trabajan 8 horas diarias.

import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Define customer and module characteristics
customers = [
    {"id": 1, "modules": [1, 2, 3]},
    {"id": 2, "modules": [1, 4, 5]},
    {"id": 3, "modules": [2, 3, 4]},
    {"id": 4, "modules": [6, 6]}  # Two requests for module 6
]

module_servers = {
    1: 2,
    2: 2,
    3: 2,
    4: 1,
    5: 1,
    6: 1
}

# Initialize simulation variables
current_time = datetime.now()
output_data = []

class Server:
    def __init__(self):
        self.busy_until = current_time
        self.current_request = None
        self.daily_capacity = timedelta(hours=8)  # Max 8 hours per day
        self.total_working_time = timedelta(hours=0)

servers = {
    1: Server(),
    2: Server(),
    3: Server(),
    4: Server(),
    5: Server(),
    6: Server()
}

# Define service time distributions (mean and std deviation)
module_service_distributions = {
    1: {"mean": 3, "std_dev": 1},
    2: {"mean": 3, "std_dev": 1},
    3: {"mean": 3, "std_dev": 1},
    4: {"mean": 4, "std_dev": 2},
    5: {"mean": 4, "std_dev": 2},
    6: {"mean": 10, "std_dev": 3}
}

# Simulate the queue and service
while current_time.time() < datetime.strptime("16:00", "%H:%M").time():  # Simulate for 8 hours
    available_servers = [module for module, server in servers.items() if server.busy_until <= current_time]

    if not available_servers:
        min_busy_until = min([server.busy_until for server in servers.values()])
        current_time = min_busy_until
        continue

    module = available_servers[0]
    for server in available_servers[1:]:
        if servers[server].busy_until < servers[module].busy_until:
            module = server

    customer = next((c for c in customers if module in c["modules"]), None)
    if customer is None:
        current_time += timedelta(seconds=1)
        continue

    start_time = current_time

    # Generate random service time based on the normal distribution
    service_time = max(np.random.normal(module_service_distributions[module]["mean"],
                                        module_service_distributions[module]["std_dev"]), 0.5)

    end_time = current_time + timedelta(minutes=service_time)

    if customer["id"] == 4:
        if servers[module].current_request is None:
            servers[module].current_request = 1
        else:
            servers[module].current_request = 2

    output_data.append({
        "Customer ID": customer["id"],
        "Module": module,
        "Start Time": start_time,
        "End Time": end_time,
        "Server": f"Server {module_servers[module]}",
        "Request": servers[module].current_request if customer["id"] == 4 else None
    })

    servers[module].busy_until = end_time
    servers[module].total_working_time += end_time - start_time
    current_time = end_time

# Create DataFrame and Excel output
df = pd.DataFrame(output_data)
file_path = "queue_output.xlsx"
df.to_excel(file_path, index=False)
print(f"Output saved to {file_path}")


Output saved to queue_output.xlsx


In [14]:
# Codigo con restriccion en los servidores: trabajan 8 horas diarias de lunes a viernes. Pdta: se queda en un bucle infinito!

import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Define customer and module characteristics
customers = [
    {"id": 1, "modules": [1, 2, 3]},
    {"id": 2, "modules": [1, 4, 5]},
    {"id": 3, "modules": [2, 3, 4]},
    {"id": 4, "modules": [6, 6]}  # Two requests for module 6
]

module_servers = {
    1: 2,
    2: 2,
    3: 2,
    4: 1,
    5: 1,
    6: 1
}

# Initialize simulation variables
current_time = datetime.now()
output_data = []

class Server:
    def __init__(self):
        self.busy_until = current_time
        self.current_request = None
        self.daily_capacity = timedelta(hours=8)  # Max 8 hours per day

servers = {
    1: Server(),
    2: Server(),
    3: Server(),
    4: Server(),
    5: Server(),
    6: Server()
}

# Define service time distributions (mean and std deviation)
module_service_distributions = {
    1: {"mean": 3, "std_dev": 1},
    2: {"mean": 3, "std_dev": 1},
    3: {"mean": 3, "std_dev": 1},
    4: {"mean": 4, "std_dev": 2},
    5: {"mean": 4, "std_dev": 2},
    6: {"mean": 10, "std_dev": 3}
}

# Simulate the queue and service
while current_time.weekday() < 5:  # Monday to Friday
    for customer in customers:
        for module in customer["modules"]:
            while servers[module].busy_until > current_time or current_time - servers[module].busy_until > servers[module].daily_capacity:
                current_time += timedelta(seconds=1)

            start_time = current_time

            # Generate random service time based on the normal distribution
            service_time = max(np.random.normal(module_service_distributions[module]["mean"],
                                                module_service_distributions[module]["std_dev"]), 0.5)

            end_time = current_time + timedelta(minutes=service_time)

            if customer["id"] == 4:
                if servers[module].current_request is None:
                    servers[module].current_request = 1
                else:
                    servers[module].current_request = 2

            output_data.append({
                "Customer ID": customer["id"],
                "Module": module,
                "Start Time": start_time,
                "End Time": end_time,
                "Server": f"Server {module_servers[module]}",
                "Request": servers[module].current_request if customer["id"] == 4 else None
            })

            servers[module].busy_until = end_time
            current_time = end_time
    current_time += timedelta(days=1)  # Move to the next day

# Create DataFrame and Excel output
df = pd.DataFrame(output_data)
file_path = "queue_output.xlsx"
df.to_excel(file_path, index=False)
print(f"Output saved to {file_path}")

KeyboardInterrupt: ignored